In [ ]:
'''
Code: Stock GUI
Name: Asha Nagireddy
Student ID: 0682057
version : 1
Date: 12/07/2019
'''
import tkinter
import tkinter.messagebox
import requests
import sys
import matplotlib.pyplot as plt
import datetime
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

#function to retrive and display stock price and trading volume...
def info():
    try:
        global sym  #declaring global variable
        sym = s1.get()   #getting input symbol string
        if sym == "":    #input from option menu..
            sym = mu1.get()
        response = requests.get("https://www.alphavantage.co/query", params = {"function": "GLOBAL_QUOTE", "symbol" : sym, "apikey" : "AYDIVYN7CLZT52IB"})
        data = response.json()
        quote = data['Global Quote']
        out_str = "Stock price: "+quote["05. price"]+ "\n Trading volume: "+quote["06. volume"]
        s2.set(out_str)   #text to display on page
    except:
        tkinter.messagebox.showinfo("Error", "incorrect stock symbol")  #error popup

#function to update stock information...        
def refresh():
    info()   

#function to plot the stocks graphs
def plot():
    if r1.get() == 1: #validating response from radio buttons for current day
        response = requests.get("https://www.alphavantage.co/query", params = {"function":"TIME_SERIES_INTRADAY","symbol":sym,"interval":"5min","apikey" : "AYDIVYN7CLZT52IB"})
        data = response.json()
        quote = data['Time Series (5min)']
        list_time = list(quote.keys())
        today = str(datetime.date.today()) #getting todays date
        X = []
        Y = []
        for i in list_time:
            temp = i.split(" ")
            if temp[0] == today: #filtering todays data
                X.append(temp[1])
                y = int(quote[i]['5. volume']) #extracting volume information
                Y.append(y)
        #labels....
        title = 'Todays Trading volume'
        x_label = 'Time'
        y_label = 'Trading Volume'
        
        
    else: #when current month radio button is selected...
        response = requests.get("https://www.alphavantage.co/query", params = {"function":"TIME_SERIES_DAILY","symbol":"MFST","apikey" : "AYDIVYN7CLZT52IB"})
        data = response.json()
        quote = data['Time Series (Daily)']
        list_time = list(quote.keys())
        today = str(datetime.date.today())
        today = today.split("-")
        X = []
        Y = []
        for i in list_time:
            temp = i.split("-")
            if temp[1] == today[1]: #filtering current months data
                X.append(i)
                y = int(quote[i]['5. volume']) #extracting volume information
                Y.append(y)
        #labels....
        title = 'Current Month Trading Volume'
        x_label = 'Date'
        y_label = 'Trading Volume'
        
    fig = plt.figure(figsize=(15,5))
    plot = fig.add_subplot(1, 1, 1)
    plot.scatter(X,Y, color = 'g')
    plot.plot(X,Y)
    plot.legend() 
    plot.set_xlabel(x_label)
    plot.set_ylabel(y_label)
    plot.set_title(title)
    plt.xticks(rotation = 40)
    canvas = FigureCanvasTkAgg(fig, window)
    canvas.get_tk_widget().grid(row=7, column=0,columnspan = 2, sticky = "wens")

#Function to exit page..
def gohome():
    global window
    window.destroy()  #sys.exit funtion kills kernal in jupiter notebook
    
#creating a tkinter window    
window = tkinter.Tk()
window.title("Stock Information") #title display
window.config(height = 500,width = 700, bg = "PeachPuff")

#displaying welcome message...
line1 = tkinter.Label(window, text = "Welcome to the Stock API! Please enter/choose a stock symbol to view the stock information", bg = "pink")
line1.grid(row = 0, column = 0, columnspan = 2, sticky = "wens")
line1.config(wraplength = 500)

line2 = tkinter.Label(window, text = "Stock symbol:")
line2.grid(row = 1,column = 0,sticky ="wens")

#taking userinput for stock symbol..
s1 = tkinter.StringVar()
entry1 = tkinter.Entry(window,textvariable = s1)
entry1.grid(row = 1, column = 1, sticky = "wens")

# option menu for selecting stock symbol...
mu1 = tkinter.StringVar()
mu1.set("MFST")
menuchoices = ["MSFT","MFST","NSE:ASHOKLEY","NSE:HDFC","NSE:SBIN","NSE:LT","NSE:ARVSMART","SOL.JO","LON:RR","GBG.L"]
menu1 = tkinter.OptionMenu(window, mu1, *menuchoices)
menu1.grid(row = 1, column = 2,sticky = "wens")

#space to display stock price and volume...
s2 = tkinter.StringVar()
s2.set("Default")
out1 = tkinter.Message(window, textvariable = s2, bg = "light blue", width = 400)
out1.grid(row = 2, column = 0,columnspan = 2, sticky = "wens")

# button to initiate info function to retrieve stock price and volume...
b1 = tkinter.Button(window, text = "Do it!", bg = "dark sea green", command = info)
b1.grid(row = 3, column = 0, sticky = "wens")

#button to trigger update event
b2 = tkinter.Button(window, text = "Update", bg = "dark sea green", command = refresh)
b2.grid(row = 3, column = 1, sticky = "wens")

#radio button to take user option for current day or month...
r1 = tkinter.IntVar()
r1.set(1)
rb1 = tkinter.Radiobutton(window, variable = r1, value = 1, text = "current day")
rb2 = tkinter.Radiobutton(window, variable = r1, value = 2, text = "current month")
rb1.grid(row = 4, column = 0, sticky = "wens")
rb2.grid(row = 4, column = 1, sticky = "wens")

#button to trigger plot function..
b3 = tkinter.Button(window, text = "Graph", bg = "turquoise", command = plot)
b3.grid(row = 5, column = 0,columnspan = 2, sticky = "wens")

#button to exit page..
b4 = tkinter.Button(window, text = "Quit", bg = "tomato", command = gohome)
b4.grid(row = 6, column = 0,columnspan = 2, sticky = "wens")

window.rowconfigure(0, weight = 1)
window.rowconfigure(1, weight = 1)
window.rowconfigure(2, weight = 1)
window.rowconfigure(3, weight = 1)
window.rowconfigure(4, weight = 1)
window.rowconfigure(5, weight = 1)
window.rowconfigure(6, weight = 1)
window.columnconfigure(0, weight = 1)
window.columnconfigure(1, weight = 1)

window.mainloop()
